In [2]:
%reload_ext autoreload
%autoreload 2
import sys
import os
import importlib
script_dir = os.getcwd()  
sys.path.append(os.path.abspath(os.path.join(script_dir, '..')))
import warnings
warnings.filterwarnings("ignore")

In [3]:
# LOOK AT NOTES TO SEE PROCESS OF TUNING HYPERPARAMETERS
import pandas as pd
from src.paths import TRANSFORMED_DATA_DIR

df = pd.read_parquet(TRANSFORMED_DATA_DIR/ "tabular_data.parquet")
df

,rides_previous_672_hour,rides_previous_671_hour,rides_previous_670_hour,rides_previous_669_hour,rides_previous_668_hour,rides_previous_667_hour,rides_previous_666_hour,rides_previous_665_hour,rides_previous_664_hour,rides_previous_663_hour,...,rides_previous_7_hour,rides_previous_6_hour,rides_previous_5_hour,rides_previous_4_hour,rides_previous_3_hour,rides_previous_2_hour,rides_previous_1_hour,pickup_hour,pickup_location_id,target_rides_next_hour
0,11.0,15.0,26.0,8.0,9.0,7.0,3.0,1.0,0.0,3.0,...,11.0,7.0,4.0,3.0,4.0,9.0,19.0,2022-01-29,4,17.0
1,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,5.0,4.0,10.0,7.0,5.0,9.0,10.0,2022-01-30,4,9.0
2,0.0,1.0,0.0,0.0,1.0,1.0,1.0,3.0,2.0,3.0,...,8.0,7.0,8.0,5.0,5.0,10.0,0.0,2022-01-31,4,3.0
3,1.0,1.0,0.0,0.0,0.0,3.0,2.0,3.0,4.0,5.0,...,3.0,16.0,7.0,1.0,0.0,1.0,3.0,2022-02-01,4,3.0
4,0.0,0.0,0.0,0.0,0.0,0.0,3.0,4.0,1.0,2.0,...,3.0,8.0,3.0,0.0,4.0,4.0,3.0,2022-02-02,4,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88289,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-12-27,199,0.0
88290,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-12-28,199,0.0
88291,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-12-29,199,0.0
88292,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-12-30,199,0.0


In [4]:
from datetime import datetime
from src.data_split import train_test_split # import func from py file

X_train, y_train, X_test, y_test = train_test_split(df, cutoff_date=datetime(2022, 6, 1, 0, 0, 0), target_column_name="target_rides_next_hour")

print(f"X_train: {X_train.shape}") # (examples-in-train, num-features)
print(f"y_train: {y_train.shape}") # (examples-in-train,) 1D beceause only one target
print(f"X_test: {X_test.shape}")   # (examples-in-test, num-features)
print(f"y_test: {y_test.shape}")   # (examples-in-test,) 1D beceause only one target
# examples-in-train != examples-in-test
# x_train
# y_train

X_train: (32226, 674)
y_train: (32226,)
X_test: (56068, 674)
y_test: (56068,)


In [6]:
import numpy as np
from sklearn.model_selection import KFold, TimeSeriesSplit
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_error
import optuna # hyperparameter optimization framework designed to automate the process of hyperparameter tuning in machine learning models. It provides an efficient way to search for the best set of hyperparameters that maximize or minimize a given objective function, such as model accuracy or loss.
from src.model import get_pipeline

# given set of HP, it trains a model and computes a validation error based on a TimeSeriesSplit
def objective(trial): # Optuna-trial
    hyperparams = {
        "metric": "mae",
        "verbose": -1,
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.2, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.2, 1.0),
        "min_child_samples": trial.suggest_int("min_child_samples", 3, 100), 
    }  # select HP

    tss = KFold(n_splits=3)  # define number of splitsm This initializes a KFold cross-validator with 3 splits, which means the data will be split into 3 parts for cross-validation.
    scores = []   # stores MAE-score of each split

    # iterate through each split in X-train, for each train-index and validation-data-index
    for train_index, val_index in tss.split(X_train):
        # split-train-data = split X-train-df from cur-train-index to end of dataframe
        # split-validation-data = split X-train-df from cur-validaiton-index to end of dataframe
        X_train_, X_val_ = X_train.iloc[train_index, :], X_train.iloc[val_index, :]
        # split-Ytrain-data = split by setting to cur-train-index of Y-train-df  
        # split-Yvalidation-data = split by setting to cur-val-index of Y-train-df  
        y_train_, y_val_ = y_train.iloc[train_index], y_train.iloc[val_index]

        
        pipeline = get_pipeline(**hyperparams)  # create pipeline-obj
        pipeline.fit(X_train_, y_train_)  # call fit on pipeline which performs defiend transformations in sequence and trains passed in model. Train on traing-split-cur-data

        y_pred = pipeline.predict(X_val_)  # compute predictions and error, on cur-validation-data for cur split
        mae = mean_absolute_error(y_val_, y_pred)
        scores.append(mae)

    return np.array(scores).mean()
        


In [7]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=5)

[I 2024-06-26 20:46:52,588] A new study created in memory with name: no-name-10cf1fd4-fcc2-414a-86e7-32fc54fa1b62
/Users/pravachanpatra/Documents/PYTHON/AI:ML:DL/Real World ML Tutorial/Taxi Demand Predictor Project/taxi_demand_predictor/src/model.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["average_rides_last_4_weeks"] = 0.25*(
/Users/pravachanpatra/Documents/PYTHON/AI:ML:DL/Real World ML Tutorial/Taxi Demand Predictor Project/taxi_demand_predictor/src/model.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-v

In [8]:
best_params = study.best_trial.params  # extract best-HP that optuna found
print(f'{best_params=}')

best_params={'num_leaves': 222, 'feature_fraction': 0.6184803512081585, 'bagging_fraction': 0.3312409733936035, 'min_child_samples': 15}


In [9]:
pipeline = get_pipeline(**best_params) # create pipeline-obj, passing in the new best found HP
pipeline.fit(X_train, y_train) # call fit on pipeline which performs defiend transformations in sequence and trains the original train-X, train-y dataframes

predictions = pipeline.predict(X_test)
test_mae = mean_absolute_error(y_test, predictions)
print(f'{test_mae=:.4f}')

[LightGBM] [Warning] feature_fraction is set=0.6184803512081585, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6184803512081585
[LightGBM] [Warning] bagging_fraction is set=0.3312409733936035, subsample=1.0 will be ignored. Current value: bagging_fraction=0.3312409733936035
[LightGBM] [Warning] feature_fraction is set=0.6184803512081585, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6184803512081585
[LightGBM] [Warning] bagging_fraction is set=0.3312409733936035, subsample=1.0 will be ignored. Current value: bagging_fraction=0.3312409733936035
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.050224 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 154527
[LightGBM] [Info] Number of data points in the train set: 32226, number of used features: 675
[LightGBM] [Info] Start training from scor

In [10]:
from src.plot import plot_one_sample

plot_one_sample(
    example_id=2979,
    features=X_test,
    targets=y_test,
    predictions=pd.Series(predictions)
)

plot_one_sample(
    example_id=1300,
    features=X_test,
    targets=y_test,
    predictions=pd.Series(predictions)
)
